In [1]:
import redis
from collections import Counter

In [2]:
CLIENTS_NUM = 2

In [3]:
default_responses_ack = {}
responses_ack = {}
client_responses = {}
for i in range(0, CLIENTS_NUM):
    default_responses_ack[f"client_{i}"] = False
    responses_ack[f"client_{i}"] = False
    client_responses[f"client_{i}"] = ''
print(responses_ack)

{'client_0': False, 'client_1': False}


In [4]:
subscriber = redis.Redis(
  host='redis-10323.c274.us-east-1-3.ec2.redns.redis-cloud.com',
  port=10323,
  password='hN34pvKTT8nrhOe3S9annZ6R6yNzY5Ia')

In [5]:
# Subscribe to the channel
pubsub = subscriber.pubsub()
pubsub.subscribe('messages')

In [6]:
def all_ack():
    state = True
    for sender in responses_ack:
        if(not responses_ack[sender]):
            return False
    return True

def reset_acks():
    responses_ack = default_responses_ack

In [7]:
def handle_aggregation():
    print("Handling aggss")
    print(client_responses)
    responses = []
    for client in client_responses:
        responses.append(client_responses[client])

    count = Counter(responses)
    most_common_num, most_common_count = count.most_common(1)[0]


    classification_result = most_common_num
    subscriber.publish('messages', f'user_response#server#{classification_result}')
    return classification_result

In [8]:
can_accept_inputs = True
# Listen for messages (this will block and wait for messages)
for message in pubsub.listen():
    message_type = message['type']
    if message['type'] == 'message':
        message = message['data'].decode()

        message_type = message.split("#")[0]
        sender = message.split("#")[1]
        message_content = message.split("#")[2]

        print(f'{message_type}@{sender}:', message_content)

        if(message_type == "input" and can_accept_inputs):
            subscriber.publish('messages', f'client_task#server#{message_content}')

        if(message_type == "classification"):
            if sender in responses_ack.keys():
                responses_ack[sender] = True
                client_responses[sender] = message_content
            
            if all_ack():
                print('All Received -> Start aggregation')
                reset_acks()
                handle_aggregation()
        
        if(message_type == "TERMINATE"):
            break
                
                


input@user: Hi. how are you ?
input@user: YOU WON $500000! YOU WON $500000! YOU WON $500000! 
client_task@server: Hi. how are you ?
client_task@server: YOU WON $500000! YOU WON $500000! YOU WON $500000! 
classification@client_0: 0
classification@client_1: 0
All Received -> Start aggregation
Handling aggss
{'client_0': '0', 'client_1': '0'}
classification@client_0: 0
All Received -> Start aggregation
Handling aggss
{'client_0': '0', 'client_1': '0'}
user_response@server: 0
classification@client_1: 0
All Received -> Start aggregation
Handling aggss
{'client_0': '0', 'client_1': '0'}
user_response@server: 0
user_response@server: 0
